In [1]:
%load_ext line_profiler

In [2]:
import types
import time
import copy

import pyinstrument
import numpy as np
import risk_ext

import torch

from env import DumbPlayer, play_games
from vpg import VPGPlayer, train

In [3]:
game_spec = dict(
    n_players=3,
    n_territories=8,
    baseline_reinforcements=3,
    n_attacks_per_turn=10,
)

In [4]:
players = [VPGPlayer()] + [DumbPlayer() for i in range(game_spec['n_players'] - 1)]
#players = [DumbPlayer()] + [DumbPlayer() for i in range(game_spec['n_players'] - 1)]
nn_player_idx = 0

In [5]:
play_games(game_spec, players, np.arange(50))

[0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 1]

In [6]:
play_games(env, players, 10)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [7]:
%lprun -f play_games -f act -f act_dumb play_games(env, players, 250)

Timer unit: 1e-06 s

Total time: 1.60376 s
File: <ipython-input-5-0de797d7b9be>
Function: act at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def act(self, player_idx, states):
     7      3323       4683.0      1.4      0.3      if states.shape[0] == 0:
     8        14         89.0      6.4      0.0          return np.array([]).reshape((0,2))
     9      3309      11263.0      3.4      0.7      state_matrix = states[:, n_max_players:].reshape((states.shape[0], n_max_territories, n_max_players+1))
    10      3309      45570.0     13.8      2.8      attack_from = (state_matrix[:,:,player_idx + 1] == 1).argmax(axis = 1)
    11                                               
    12      3309      42091.0     12.7      2.6      states_torch = torch.as_tensor(states, dtype=torch.float32)
    13      3309    1471464.0    444.7     91.8      attack_to = self.get_action(states_torch)
    14                             

In [25]:
%lprun -f env.faceoff -f env.play_game -f VPGPlayer.act env.faceoff([VPGPlayer(env)] + [DumbPlayer() for i in range(game_spec['n_players'] - 1)], 250)

/home/tbent/Dropbox/active/rlrisk/rlrisk/vpg.py:69: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1614378073166/work/torch/csrc/utils/tensor_numpy.cpp:143.)
  attack_to = self.get_action(torch.as_tensor(state_vec, dtype=torch.float32))


Timer unit: 1e-06 s

Total time: 15.1733 s
File: /home/tbent/Dropbox/active/rlrisk/rlrisk/env.py
Function: faceoff at line 67

Line #      Hits         Time  Per Hit   % Time  Line Contents
    67                                               def faceoff(self, players, n_games):
    68         1         14.0     14.0      0.0          winners = np.empty(n_games, dtype=np.int32)
    69       251        168.0      0.7      0.0          for i in range(n_games):
    70       250   15173017.0  60692.1    100.0              winners[i] = self.play_game(players)
    71         1        140.0    140.0      0.0          winner_counts = np.unique(winners, return_counts=True)
    72         1         10.0     10.0      0.0          return winners, winner_counts[0], winner_counts[1] / n_games

Total time: 14.563 s
File: /home/tbent/Dropbox/active/rlrisk/rlrisk/env.py
Function: play_game at line 74

Line #      Hits         Time  Per Hit   % Time  Line Contents
    74                                

In [6]:
from pyinstrument import Profiler
profiler = Profiler()
profiler.start()

#play_games(env, players, 50)
#play_games(env, players, 50)
env.faceoff([VPGPlayer(env)] + [DumbPlayer() for i in range(game_spec['n_players'] - 1)], 50)
#train(env, players, 2, 20000, [0])

profiler.stop()
print(profiler.output_text(unicode=True, color=True))

/home/tbent/Dropbox/active/rlrisk/rlrisk/vpg.py:69: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1614378073166/work/torch/csrc/utils/tensor_numpy.cpp:143.)
  attack_to = self.get_action(torch.as_tensor(state_vec, dtype=torch.float32))



  _     ._   __/__   _ _  _  _ _/_   Recorded: 22:34:21  Samples:  2732
 /_//_/// /_\ / //_// / //_'/ //     Duration: 2.756     CPU time: 2.756
/   _/                      v3.4.1

Program: /home/tbent/.miniconda3/envs/rlrisk/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/tbent/.local/share/jupyter/runtime/kernel-948caa3d-9119-4b9d-a4a7-2469db6d47c6.json

2.755 run_code  IPython/core/interactiveshell.py:3396
└─ 2.755 <module>  <ipython-input-6-82be53c98072>:7
   └─ 2.754 faceoff  env.py:67
      └─ 2.754 play_game  env.py:74
         ├─ 2.040 act  vpg.py:66
         │  ├─ 1.912 get_action  vpg.py:59
         │  │  ├─ 1.557 get_policy  vpg.py:55
         │  │  │  ├─ 0.777 __init__  torch/distributions/categorical.py:49
         │  │  │  │     [32 frames hidden]  torch, <built-in>
         │  │  │  └─ 0.764 _call_impl  torch/nn/modules/module.py:866
         │  │  │        [36 frames hidden]  torch, <built-in>
         │  │  ├─ 0.252 sample  torch/distributions/categorical.p